# read in and set up all the data 

In [ ]:
# Import some necessary modules
import os
import sys
import pickle
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer, Chain, Truth, PlotConfig     
import warnings
warnings.filterwarnings('ignore')

sys.path.append("../../Barry/")     # Change this so that it points to where you have Barry installed

#from barry.samplers import DynestySampler
from barry.samplers import NautilusSampler
from barry.config import setup
from barry.models import PowerBeutler2017, CorrBeutler2017
from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4
from barry.datasets.dataset_correlation_function import CorrelationFunction_DESI_KP4
from barry.fitter import Fitter
from barry.models.model import Correction # class for applying corrections to the likelihood function 
from barry.utils import weighted_avg_and_cov # function for getting avg and covariance 


path_p1 = '/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/plots/'

dat = '_xi' 

pfns = []
#listnames = ['BGS_z01_04', 'LRGs_z04_06', 'LRGs_z06_08', 'LRGs_z08_11', 'ELGs_z11_16', 'QSOs_z08_21']
listnames = ['BGS_z01_04', 'LRGs_z04_06', 'LRGs_z06_08', 'ELGsLRGscombined_z08_11', 'ELGs_z11_16', 'QSOs_z08_21']
for i in listnames:
    pfns.append(path_p1 + 'desi_kp4_SecondGen_' + i + dat + '-reducedcov/output/desi_kp4_SecondGen_' + i + dat + '-reducedcov.fitter.pkl')

fits_chains = {}
fits_summaries = {}

for i in listnames:
    for p in [' post recon']:# , ' pre recon']:
        for j in [' spline', ' poly']:
            for b in [' phase shift fit', ' standard fit']:
                fits_chains[i + p + j + b] = ChainConsumer()                      
                
for i,p in enumerate(pfns): 

    with open(p, 'rb') as pickle_file:
        fitter = pickle.load(pickle_file)

    for posterior, weight, chain, evidence, model, data, extra in fitter.load():
        
        df = pd.DataFrame(chain, columns=model.get_labels())
        
        extraname = ''
        
        if 'mean' in extra['name']:
            extraname = ' mean'
        else:
            extraname = extra['name'].split(' ')[-1]
            
        recon = ''
        if model.recon:
            recon = ' post recon'
        else:
            continue 
            recon = ' pre recon' 
            
        beta = ''
        if '$\\beta_{\\phi(N_{\\mathrm{eff}})}$' in model.get_labels():
            beta = ' phase shift fit'
        else:    
            beta = ' standard fit' 
            
        
        fits_chains[listnames[i] + recon + ' ' + model.broadband_type + beta].add_chain(
            Chain(samples=df, weights=weight, name=extraname, plot_contour=True, plot_point=False, show_as_1d_prior=False))
        
for i in listnames:
    for p in [' post recon']:#, ' pre recon']:
        for j in [' spline', ' poly']:
            for b in [' phase shift fit', ' standard fit']:
                fits_summaries[i + p + j + b] = fits_chains[i + p + j + b].analysis.get_summary() 
                

                

for p in [' post recon']:#, ' pre recon']:
    for j in [' spline', ' poly']:
        fits_chains['combined fits' + p  + j + ' phase shift fit'] = ChainConsumer() 

# main = '/global/u1/a/abbew25/barryrepo/Barry/cosmodesi_KP4ELG_examplecode_make_picklefiles/combinedfits_secondgen_mocks_v1_2/'
main = '/pscratch/sd/a/abbew25/combinedfits_secondgen_mocks_v1_2_shuffled/'

# count = 0
# for i in np.arange(52,77+1):#(104, 129+1):
#     name = ' mean' if i == 52 else str(count)
#     count += 1 
#     print(name)
#     df = pd.read_csv(main + dat[1:] + "_post_spline_" + str(i) + ".csv", index_col=0)
#     fits_chains['combined fits post recon spline phase shift fit'].add_chain(Chain(samples=df, name=name, plot_contour=True, plot_point=False, show_as_1d_prior=False))
# fits_summaries['combined fits post recon spline phase shift fit'] = fits_chains['combined fits post recon spline phase shift fit'].analysis.get_summary() 

# count = 0
# for i in np.arange(78, 103+1):#(156, 181+1):
#     name = ' mean' if i == 78 else str(count)
#     count += 1 
#     df = pd.read_csv(main + dat[1:] + "_post_poly_" + str(i) + ".csv", index_col=0)
#     fits_chains['combined fits post recon poly phase shift fit'].add_chain(Chain(samples=df, name=name, plot_contour=True, plot_point=False, show_as_1d_prior=False))
# fits_summaries['combined fits post recon poly phase shift fit'] = fits_chains['combined fits post recon poly phase shift fit'].analysis.get_summary()
    
# count = 0
# for i in np.arange(0, 25+1):
#     name = ' mean' if i == 0 else str(count)
#     count += 1 
#     df = pd.read_csv(main + dat[1:] + "_pre_spline_" + str(i) + ".csv", index_col=0)
#     fits_chains['combined fits pre recon spline phase shift fit'].add_chain(Chain(samples=df, name=name, plot_contour=True, plot_point=False, show_as_1d_prior=False))
# fits_summaries['combined fits pre recon spline phase shift fit'] = fits_chains['combined fits pre recon spline phase shift fit'].analysis.get_summary() 

# count = 0
# for i in np.arange(26, 51+1):#(52, 77+1):
#     name = ' mean' if i == 26 else str(count)
#     count += 1
#     df = pd.read_csv(main + dat[1:] + "_pre_poly_" + str(i) + ".csv", index_col=0)
#     fits_chains['combined fits pre recon poly phase shift fit'].add_chain(Chain(samples=df, name=name, plot_contour=True, plot_point=False, show_as_1d_prior=False))
# fits_summaries['combined fits pre recon poly phase shift fit'] = fits_chains['combined fits pre recon poly phase shift fit'].analysis.get_summary() 



# Plot fits for alpha, epsilon for correlation function

In [ ]:
recon = 'post'
fit = 'phase shift fit' 
plt.rcParams["figure.figsize"] = [15, 20]
f, axes = plt.subplots(10, 1, sharex=True)
axes[0].set_xlim([-1, 26])
axes[0].set_title(recon + ' recon, xi, ' + fit)

axes[0].hlines(1.0, -1, 26, linestyle='--', color='grey')

axes[1].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[2].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[3].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[4].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[5].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[6].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[7].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[8].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[9].hlines(1.0, -1, 26, linestyle='--', color='grey')

alpha_BGS = {'spline': [], 'poly': []}
axes[0].set_ylabel(r'$\alpha_{BGS}$')

alpha_LRG1 = {'spline': [], 'poly': []}
epsilon_LRG1 = {'spline': [], 'poly': []}
axes[1].set_ylabel(r'$\alpha_{LRG1}$')
axes[2].set_ylabel(r'$\epsilon_{LRG1}$')

alpha_LRG2 = {'spline': [], 'poly': []}
epsilon_LRG2 = {'spline': [], 'poly': []}
axes[3].set_ylabel(r'$\alpha_{LRG2}$')
axes[4].set_ylabel(r'$\epsilon_{LRG2}$')

alpha_LRG3ELG1 = {'spline': [], 'poly': []}
epsilon_LRG3ELG1 = {'spline': [], 'poly': []}
axes[5].set_ylabel(r'$\alpha_{LRG3ELG1}$')
axes[6].set_ylabel(r'$\epsilon_{LRG3ELG1}$')

alpha_ELG2 = {'spline': [], 'poly': []}
epsilon_ELG2 = {'spline': [], 'poly': []}
axes[7].set_ylabel(r'$\alpha_{ELG2}$')
axes[8].set_ylabel(r'$\epsilon_{ELG2}$')

alpha_QSO = {'spline': [], 'poly': []}
axes[9].set_ylabel(r'$\alpha_{QSO}$')

count = 0
for item in fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit]:
    
    for broadband in ['spline', 'poly']:
        
        delta = 0.0 if broadband == 'spline' else 0.2
        color = 'blue' if broadband == 'spline' else 'green'
        if count == 0:
            color  = 'k' if broadband == 'spline' else 'r'
    
        x = fits_summaries['BGS_z01_04 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[0].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[0].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
        alpha_BGS[broadband].append(x['$\\alpha$'].center)

        # -----------------------------

        x = fits_summaries['LRGs_z04_06 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[1].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[1].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_LRG1[broadband].append(x['$\\alpha$'].center)
        
        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[2].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[2].scatter(count+delta, x['$\\epsilon$'].center, color=color, alpha=0.5)
        epsilon_LRG1[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------

        x = fits_summaries['LRGs_z06_08 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[3].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[3].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_LRG2[broadband].append(x['$\\alpha$'].center)

        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[4].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[4].scatter(count+delta, x['$\\epsilon$'].center, color=color, alpha=0.5)
            
        epsilon_LRG2[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------

        x = fits_summaries['ELGsLRGscombined_z08_11 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[5].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[5].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_LRG3ELG1[broadband].append(x['$\\alpha$'].center)

        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[6].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[6].scatter(count+delta, x['$\\epsilon$'].center, color=color, alpha=0.5)
            
        epsilon_LRG3ELG1[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------

        x = fits_summaries['ELGs_z11_16 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[7].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[7].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_ELG2[broadband].append(x['$\\alpha$'].center)

        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[8].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[8].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        epsilon_ELG2[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------
    
        x = fits_summaries['QSOs_z08_21 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[9].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[9].scatter(count, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_QSO[broadband].append(x['$\\alpha$'].center)

    count += 1 
    
for i, dictt in enumerate([alpha_BGS, 
                           alpha_LRG1, epsilon_LRG1, 
                           alpha_LRG2, epsilon_LRG2, 
                           alpha_LRG3ELG1, epsilon_LRG3ELG1,
                           alpha_ELG2, epsilon_ELG2,
                           alpha_QSO
                          ]): 
    
    for broadband in ['spline', 'poly']: 
        
        color = 'blue' if broadband == 'spline' else 'green'
        
        axes[i].fill_between([-1, 26], np.mean(dictt[broadband][1:])-np.std(dictt[broadband][1:])/5.0, 
            np.mean(dictt[broadband][1:])+np.std(dictt[broadband][1:])/5.0, alpha=0.1, color=color)



# plots fits for beta phase shift for correlation function

In [ ]:
recon = 'post'
fit = 'phase shift fit' 
plt.rcParams["figure.figsize"] = [15, 20]
f, axes = plt.subplots(7, 1, sharex=True)
axes[0].set_xlim([-1, 26])
axes[0].set_title(recon + ' recon, xi, ' + fit)

axes[0].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[1].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[2].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[3].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[4].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[5].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[6].hlines(1.0, -1, 26, linestyle='--', color='grey')


beta_BGS = {'spline': [], 'poly': []}
axes[0].set_ylabel(r'$\beta_{BGS, \phi}$')

beta_LRG1 = {'spline': [], 'poly': []}
axes[1].set_ylabel(r'$\beta_{LRG1, \phi}$')

beta_LRG2 = {'spline': [], 'poly': []}
axes[2].set_ylabel(r'$\beta_{LRG2, \phi}$')

beta_LRG3ELG1 = {'spline': [], 'poly': []}
axes[3].set_ylabel(r'$\beta_{LRG3ELG1, \phi}$')

beta_ELG2 = {'spline': [], 'poly': []}
axes[4].set_ylabel(r'$\beta_{ELG2, \phi}$')

beta_QSO = {'spline': [], 'poly': []}
axes[5].set_ylabel(r'$\beta_{QSO, \phi}$')

beta_combined = {'spline': [], 'poly': []}
axes[6].set_ylabel(r'$\beta_{combined, \phi}$')


beta_code = '$\\beta_{\\phi(N_{\\mathrm{eff}})}$'


count = 0
for item in fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit]:
    
    for broadband in ['spline', 'poly']:
        
        delta = 0.0 if broadband == 'spline' else 0.2
        color = 'blue' if broadband == 'spline' else 'green'
        if count == 0:
            color  = 'k' if broadband == 'spline' else 'r'
    
        x = fits_summaries['BGS_z01_04 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[0].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[0].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_BGS[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['LRGs_z04_06 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[1].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[1].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_LRG1[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['LRGs_z06_08 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[2].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[2].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_LRG2[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['ELGsLRGscombined_z08_11 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[3].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[3].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_LRG3ELG1[broadband].append(x[beta_code].center)

        # -----------------------------
    
        x = fits_summaries['ELGs_z11_16 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[4].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[4].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_ELG2[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['QSOs_z08_21 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[5].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[5].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_QSO[broadband].append(x[beta_code].center)

        # -----------------------------
        
        # x = fits_summaries['combined fits ' + recon + ' recon ' + broadband + ' ' + fit][item if item == ' mean' else str(int(item)+1)]
        # beta_code_comb = '$\\beta_{N_{\\mathrm{eff}}}$'
        # if x[beta_code_comb].lower is not None and x[beta_code_comb].upper is not None:
        #     axes[6].errorbar(count+delta, x[beta_code_comb].center, [[abs(x[beta_code_comb].lower-x[beta_code_comb].center)], 
        #                                                              [abs(x[beta_code_comb].upper-x[beta_code_comb].center)]], 
        #                                                              color=color, fmt = 'o', alpha=0.5)
        # else:
        #     axes[6].scatter(count+delta, x[beta_code_comb].center, color=color, alpha=0.5)
        # beta_combined[broadband].append(x[beta_code_comb].center)

 
    count += 1 
        
for i, dictt in enumerate([beta_BGS, 
                           beta_LRG1, 
                           beta_LRG2,  
                           beta_LRG3ELG1, 
                           beta_ELG2, 
                           beta_QSO,
                           beta_combined
                          ]): 
    
    for broadband in ['spline', 'poly']: 
        
        color = 'blue' if broadband == 'spline' else 'green'
        
        axes[i].fill_between([-1, 26], np.mean(dictt[broadband][1:])-np.std(dictt[broadband][1:])/5.0, 
            np.mean(dictt[broadband][1:])+np.std(dictt[broadband][1:])/5.0, alpha=0.1, color=color)
        axes[i].fill_between([-1, 26], np.mean(dictt[broadband][1:])-np.std(dictt[broadband][1:]), 
            np.mean(dictt[broadband][1:])+np.std(dictt[broadband][1:]), alpha=0.1, color='purple' if broadband=='spline' else 'orange')



# plot fits for alpha, epsilon for power spectrum 

In [ ]:
recon = 'post'
fit = 'phase shift fit' 
plt.rcParams["figure.figsize"] = [15, 20]
f, axes = plt.subplots(10, 1, sharex=True)
axes[0].set_xlim([-1, 26])
axes[0].set_title(recon + ' recon, pk, ' + fit)

axes[0].hlines(1.0, -1, 26, linestyle='--', color='grey')

axes[1].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[2].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[3].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[4].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[5].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[6].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[7].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[8].hlines(0.0, -1, 26, linestyle='--', color='grey')

axes[9].hlines(1.0, -1, 26, linestyle='--', color='grey')

alpha_BGS = {'spline': [], 'poly': []}
axes[0].set_ylabel(r'$\alpha_{BGS}$')

alpha_LRG1 = {'spline': [], 'poly': []}
epsilon_LRG1 = {'spline': [], 'poly': []}
axes[1].set_ylabel(r'$\alpha_{LRG1}$')
axes[2].set_ylabel(r'$\epsilon_{LRG1}$')

alpha_LRG2 = {'spline': [], 'poly': []}
epsilon_LRG2 = {'spline': [], 'poly': []}
axes[3].set_ylabel(r'$\alpha_{LRG2}$')
axes[4].set_ylabel(r'$\epsilon_{LRG2}$')

alpha_LRG3 = {'spline': [], 'poly': []}
epsilon_LRG3 = {'spline': [], 'poly': []}
axes[5].set_ylabel(r'$\alpha_{LRG3}$')
axes[6].set_ylabel(r'$\epsilon_{LRG3}$')

alpha_ELG2 = {'spline': [], 'poly': []}
epsilon_ELG2 = {'spline': [], 'poly': []}
axes[7].set_ylabel(r'$\alpha_{ELG2}$')
axes[8].set_ylabel(r'$\epsilon_{ELG2}$')

alpha_QSO = {'spline': [], 'poly': []}
axes[9].set_ylabel(r'$\alpha_{QSO}$')

count = 0
for item in fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit]:
    
    for broadband in ['spline', 'poly']:
        
        delta = 0.0 if broadband == 'spline' else 0.2
        color = 'blue' if broadband == 'spline' else 'green'
        if count == 0:
            color  = 'k' if broadband == 'spline' else 'r'
    
        x = fits_summaries['BGS_z01_04 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[0].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[0].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
        alpha_BGS[broadband].append(x['$\\alpha$'].center)

        # -----------------------------

        x = fits_summaries['LRGs_z04_06 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[1].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[1].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_LRG1[broadband].append(x['$\\alpha$'].center)
        
        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[2].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[2].scatter(count+delta, x['$\\epsilon$'].center, color=color, alpha=0.5)
        epsilon_LRG1[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------

        x = fits_summaries['LRGs_z06_08 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[3].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[3].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_LRG2[broadband].append(x['$\\alpha$'].center)

        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[4].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[4].scatter(count+delta, x['$\\epsilon$'].center, color=color, alpha=0.5)
            
        epsilon_LRG2[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------

        x = fits_summaries['LRGs_z08_11 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[5].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[5].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_LRG3[broadband].append(x['$\\alpha$'].center)

        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[6].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[6].scatter(count+delta, x['$\\epsilon$'].center, color=color, alpha=0.5)
            
        epsilon_LRG3[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------

        x = fits_summaries['ELGs_z11_16 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[7].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[7].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_ELG2[broadband].append(x['$\\alpha$'].center)

        if x['$\\epsilon$'].lower is not None and x['$\\epsilon$'].upper is not None:
            axes[8].errorbar(count+delta, x['$\\epsilon$'].center, [[abs(x['$\\epsilon$'].lower-x['$\\epsilon$'].center)], [abs(x['$\\epsilon$'].upper-x['$\\epsilon$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[8].scatter(count+delta, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        epsilon_ELG2[broadband].append(x['$\\epsilon$'].center)

        # -----------------------------
    
        x = fits_summaries['QSOs_z08_21 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x['$\\alpha$'].lower is not None and x['$\\alpha$'].upper is not None:
            axes[9].errorbar(count+delta, x['$\\alpha$'].center, [[abs(x['$\\alpha$'].lower-x['$\\alpha$'].center)], [abs(x['$\\alpha$'].upper-x['$\\alpha$'].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[9].scatter(count, x['$\\alpha$'].center, color=color, alpha=0.5)
            
        alpha_QSO[broadband].append(x['$\\alpha$'].center)

    count += 1 
    
for i, dictt in enumerate([alpha_BGS, 
                           alpha_LRG1, epsilon_LRG1, 
                           alpha_LRG2, epsilon_LRG2, 
                           alpha_LRG3, epsilon_LRG3,
                           alpha_ELG2, epsilon_ELG2,
                           alpha_QSO
                          ]): 
    
    for broadband in ['spline', 'poly']: 
        
        color = 'blue' if broadband == 'spline' else 'green'
        
        axes[i].fill_between([-1, 26], np.mean(dictt[broadband][1:])-np.std(dictt[broadband][1:])/5.0, 
            np.mean(dictt[broadband][1:])+np.std(dictt[broadband][1:])/5.0, alpha=0.1, color=color)



# plot fits for beta phase-shift, for power spectrum 

In [ ]:
recon = 'post'
fit = 'phase shift fit' 
plt.rcParams["figure.figsize"] = [15, 20]
f, axes = plt.subplots(7, 1, sharex=True)
axes[0].set_xlim([-1, 26])
axes[0].set_title(recon + ' recon, pk, ' + fit)

axes[0].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[1].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[2].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[3].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[4].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[5].hlines(1.0, -1, 26, linestyle='--', color='grey')
axes[6].hlines(1.0, -1, 26, linestyle='--', color='grey')


beta_BGS = {'spline': [], 'poly': []}
axes[0].set_ylabel(r'$\beta_{BGS, \phi}$')

beta_LRG1 = {'spline': [], 'poly': []}
axes[1].set_ylabel(r'$\beta_{LRG1, \phi}$')

beta_LRG2 = {'spline': [], 'poly': []}
axes[2].set_ylabel(r'$\beta_{LRG2, \phi}$')

beta_LRG3 = {'spline': [], 'poly': []}
axes[3].set_ylabel(r'$\beta_{LRG3ELG1, \phi}$')

beta_ELG2 = {'spline': [], 'poly': []}
axes[4].set_ylabel(r'$\beta_{ELG2, \phi}$')

beta_QSO = {'spline': [], 'poly': []}
axes[5].set_ylabel(r'$\beta_{QSO, \phi}$')

beta_combined = {'spline': [], 'poly': []}
axes[6].set_ylabel(r'$\beta_{combined, \phi}$')


beta_code = '$\\beta_{\\phi(N_{\\mathrm{eff}})}$'


count = 0
for item in fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit]:
    
    for broadband in ['spline', 'poly']:
        
        delta = 0.0 if broadband == 'spline' else 0.2
        color = 'blue' if broadband == 'spline' else 'green'
        if count == 0:
            color  = 'k' if broadband == 'spline' else 'r'
    
        x = fits_summaries['BGS_z01_04 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[0].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[0].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_BGS[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['LRGs_z04_06 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[1].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[1].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_LRG1[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['LRGs_z06_08 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[2].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[2].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_LRG2[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['LRGs_z08_11 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[3].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[3].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_LRG3[broadband].append(x[beta_code].center)

        # -----------------------------
    
        x = fits_summaries['ELGs_z11_16 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[4].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[4].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_ELG2[broadband].append(x[beta_code].center)

        # -----------------------------

        x = fits_summaries['QSOs_z08_21 ' + recon + ' recon ' + broadband + ' ' + fit][item]
        if x[beta_code].lower is not None and x[beta_code].upper is not None:
            axes[5].errorbar(count+delta, x[beta_code].center, [[abs(x[beta_code].lower-x[beta_code].center)], [abs(x[beta_code].upper-x[beta_code].center)]], 
                             color=color, fmt = 'o', alpha=0.5)
        else:
            axes[5].scatter(count+delta, x[beta_code].center, color=color, alpha=0.5)
        beta_QSO[broadband].append(x[beta_code].center)

        # -----------------------------
        
        # x = fits_summaries['combined fits ' + recon + ' recon ' + broadband + ' ' + fit][item if item == ' mean' else str(int(item)+1)]
        # beta_code_comb = '$\\beta_{N_{\\mathrm{eff}}}$'
        # if x[beta_code_comb].lower is not None and x[beta_code_comb].upper is not None:
        #     axes[6].errorbar(count+delta, x[beta_code_comb].center, [[abs(x[beta_code_comb].lower-x[beta_code_comb].center)], 
        #                                                              [abs(x[beta_code_comb].upper-x[beta_code_comb].center)]], 
        #                                                              color=color, fmt = 'o', alpha=0.5)
        # else:
        #     axes[6].scatter(count+delta, x[beta_code_comb].center, color=color, alpha=0.5)
        # beta_combined[broadband].append(x[beta_code_comb].center)

 
    count += 1 
        
for i, dictt in enumerate([beta_BGS, 
                           beta_LRG1, 
                           beta_LRG2,  
                           beta_LRG3, 
                           beta_ELG2, 
                           beta_QSO,
                           beta_combined
                          ]): 
    
    for broadband in ['spline', 'poly']: 
        
        color = 'blue' if broadband == 'spline' else 'green'
        
        axes[i].fill_between([-1, 26], np.mean(dictt[broadband][1:])-np.std(dictt[broadband][1:])/5.0, 
            np.mean(dictt[broadband][1:])+np.std(dictt[broadband][1:])/5.0, alpha=0.1, color=color)
        axes[i].fill_between([-1, 26], np.mean(dictt[broadband][1:])-np.std(dictt[broadband][1:]), 
            np.mean(dictt[broadband][1:])+np.std(dictt[broadband][1:]), alpha=0.1, color='purple' if broadband=='spline' else 'orange')


# plot comparison of parameters (alpha, epsilon) for spline and poly fits (power spectrum/correlation function)

In [ ]:
recon = 'post'
fit = 'standard fit' 
plt.rcParams["figure.figsize"] = [15, 20]
f, axes = plt.subplots(10, 1, sharex=True)
axes[0].set_xlim([-1, 26])
axes[0].set_title(recon + ' recon, ' + dat[1:] + ', ' + fit + ', diff spline - poly')

axes[0].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[1].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[2].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[3].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[4].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[5].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[6].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[7].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[8].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[9].hlines(0.0, -1, 26, linestyle='--', color='grey')

alpha_BGS = []
axes[0].set_ylabel(r'$\alpha_{BGS}$')

alpha_LRG1 = []
epsilon_LRG1 = []
axes[1].set_ylabel(r'$\alpha_{LRG1}$')
axes[2].set_ylabel(r'$\epsilon_{LRG1}$')

alpha_LRG2 = []
epsilon_LRG2 = []
axes[3].set_ylabel(r'$\alpha_{LRG2}$')
axes[4].set_ylabel(r'$\epsilon_{LRG2}$')

alpha_LRG3 = []
epsilon_LRG3 = []
alpha_LRG3ELG1 = []
epsilon_LRG3ELG1 = []
axes[5].set_ylabel(r'$\alpha_{LRG3}$')
axes[6].set_ylabel(r'$\epsilon_{LRG3}$')
if dat == '_xi':
    axes[5].set_ylabel(r'$\alpha_{LRG3ELG1}$')
    axes[6].set_ylabel(r'$\epsilon_{LRG3ELG1}$')

alpha_ELG2 = []
epsilon_ELG2 = []
axes[7].set_ylabel(r'$\alpha_{ELG2}$')
axes[8].set_ylabel(r'$\epsilon_{ELG2}$')

alpha_QSO = []
axes[9].set_ylabel(r'$\alpha_{QSO}$')

count = 0
for item in fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit]:
    
    delta = 0.0
    color = 'blue' 
    if count == 0:
        color  = 'k' 

    x0 = fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['BGS_z01_04 ' + recon + ' recon poly ' + fit][item]
    
    axes[0].scatter(count+delta, x0['$\\alpha$'].center-x1['$\\alpha$'].center, color=color, alpha=0.5)
    alpha_BGS.append(x0['$\\alpha$'].center-x1['$\\alpha$'].center)

    # -----------------------------

    x0 = fits_summaries['LRGs_z04_06 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['LRGs_z04_06 ' + recon + ' recon poly ' + fit][item]
    
    axes[1].scatter(count+delta, x0['$\\alpha$'].center-x1['$\\alpha$'].center, color=color, alpha=0.5)

    alpha_LRG1.append(x0['$\\alpha$'].center-x1['$\\alpha$'].center)

    axes[2].scatter(count+delta, x0['$\\epsilon$'].center-x1['$\\epsilon$'].center, color=color, alpha=0.5)
    epsilon_LRG1.append(x0['$\\epsilon$'].center-x1['$\\epsilon$'].center)

    # -----------------------------

    x0 = fits_summaries['LRGs_z06_08 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['LRGs_z06_08 ' + recon + ' recon poly ' + fit][item]
    
    axes[3].scatter(count+delta, x0['$\\alpha$'].center-x1['$\\alpha$'].center, color=color, alpha=0.5)

    alpha_LRG2.append(x0['$\\alpha$'].center-x1['$\\alpha$'].center)

    axes[4].scatter(count+delta, x0['$\\epsilon$'].center-x1['$\\epsilon$'].center, color=color, alpha=0.5)

    epsilon_LRG2.append(x0['$\\epsilon$'].center-x1['$\\epsilon$'].center)

    # -----------------------------

    if dat == '_pk':
        x0 = fits_summaries['LRGs_z08_11 ' + recon + ' recon spline ' + fit][item]
        x1 = fits_summaries['LRGs_z08_11 ' + recon + ' recon poly ' + fit][item]

        axes[5].scatter(count+delta, x0['$\\alpha$'].center-x1['$\\alpha$'].center, color=color, alpha=0.5)

        alpha_LRG3.append(x0['$\\alpha$'].center-x1['$\\alpha$'].center)

        axes[6].scatter(count+delta, x0['$\\epsilon$'].center-x1['$\\epsilon$'].center, color=color, alpha=0.5)

        epsilon_LRG3.append(x0['$\\epsilon$'].center-x1['$\\epsilon$'].center)
    else: 
        x0 = fits_summaries['ELGsLRGscombined_z08_11 ' + recon + ' recon spline ' + fit][item]
        x1 = fits_summaries['ELGsLRGscombined_z08_11 ' + recon + ' recon poly ' + fit][item]

        axes[5].scatter(count+delta, x0['$\\alpha$'].center-x1['$\\alpha$'].center, color=color, alpha=0.5)

        alpha_LRG3ELG1.append(x0['$\\alpha$'].center-x1['$\\alpha$'].center)

        axes[6].scatter(count+delta, x0['$\\epsilon$'].center-x1['$\\epsilon$'].center, color=color, alpha=0.5)

        epsilon_LRG3ELG1.append(x0['$\\epsilon$'].center-x1['$\\epsilon$'].center)
    # -----------------------------
    
    x0 = fits_summaries['ELGs_z11_16 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['ELGs_z11_16 ' + recon + ' recon poly ' + fit][item]
    
    axes[7].scatter(count+delta, x0['$\\alpha$'].center-x1['$\\alpha$'].center, color=color, alpha=0.5)

    alpha_ELG2.append(x0['$\\alpha$'].center-x1['$\\alpha$'].center)

    axes[8].scatter(count+delta, x0['$\\epsilon$'].center-x1['$\\epsilon$'].center, color=color, alpha=0.5)

    epsilon_ELG2.append(x0['$\\epsilon$'].center-x1['$\\epsilon$'].center)
    
    # -----------------------------

    x0 = fits_summaries['QSOs_z08_21 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['QSOs_z08_21 ' + recon + ' recon poly ' + fit][item]
    
    axes[9].scatter(count+delta, x0['$\\alpha$'].center-x1['$\\alpha$'].center, color=color, alpha=0.5)

    alpha_QSO.append(x0['$\\alpha$'].center-x1['$\\alpha$'].center)

    count += 1 
    
listdat = [alpha_BGS, alpha_LRG1, epsilon_LRG1, alpha_LRG2, epsilon_LRG2, alpha_LRG3, epsilon_LRG3, alpha_ELG2, epsilon_ELG2, alpha_QSO]
if dat == '_xi': 
    listdat = [alpha_BGS, alpha_LRG1, epsilon_LRG1, alpha_LRG2, epsilon_LRG2, alpha_LRG3ELG1, epsilon_LRG3ELG1, alpha_ELG2, epsilon_ELG2, alpha_QSO]
    
for i, dictt in enumerate(listdat): 
    
    axes[i].fill_between([-1, 26], np.mean(dictt[1:])-np.std(dictt[1:])/5.0, 
            np.mean(dictt[1:])+np.std(dictt[1:])/5.0, alpha=0.1, color='blue')




# plot comparisons of parameters (beta) for spline and poly fits (power spectrum/ correlation function)

In [ ]:
recon = 'pre'
fit = 'phase shift fit' 
plt.rcParams["figure.figsize"] = [15, 20]
f, axes = plt.subplots(7, 1, sharex=True)
axes[0].set_xlim([-1, 26])
axes[0].set_title(recon + ' recon, ' + dat[1:] + ', ' + fit + ', diff spline - poly')

axes[0].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[1].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[2].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[3].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[4].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[5].hlines(0.0, -1, 26, linestyle='--', color='grey')
axes[6].hlines(0.0, -1, 26, linestyle='--', color='grey')

beta_BGS = []
axes[0].set_ylabel(r'$\beta_{BGS}$')

beta_LRG1 = []
axes[1].set_ylabel(r'$\beta_{LRG1}$')

beta_LRG2 = []
axes[2].set_ylabel(r'$\beta_{LRG2}$')

beta_LRG3 = []
beta_LRG3ELG1 = [] 
axes[3].set_ylabel(r'$\beta_{LRG3}$')
if dat == '_xi': 
    axes[3].set_ylabel(r'$\beta_{LRG3ELG1}$')

beta_ELG2 = []
axes[4].set_ylabel(r'$\beta_{ELG2}$')

beta_QSO = []
axes[5].set_ylabel(r'$\beta_{QSO}$')

beta_combined = []
axes[6].set_ylabel(r'$\beta_{combined}$')

beta_code = '$\\beta_{\\phi(N_{\\mathrm{eff}})}$'
beta_code_comb = '$\\beta_{N_{\\mathrm{eff}}}$'

count = 0
for item in fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit]:
    
    delta = 0.0
    color = 'blue' 
    if count == 0:
        color  = 'k' 

    x0 = fits_summaries['BGS_z01_04 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['BGS_z01_04 ' + recon + ' recon poly ' + fit][item]
    
    axes[0].scatter(count+delta, x0[beta_code].center-x1[beta_code].center, color=color, alpha=0.5)
    beta_BGS.append(x0[beta_code].center-x1[beta_code].center)

    # -----------------------------

    x0 = fits_summaries['LRGs_z04_06 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['LRGs_z04_06 ' + recon + ' recon poly ' + fit][item]
    
    axes[1].scatter(count+delta, x0[beta_code].center-x1[beta_code].center, color=color, alpha=0.5)
    beta_LRG1.append(x0[beta_code].center-x1[beta_code].center)
    # -----------------------------

    x0 = fits_summaries['LRGs_z06_08 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['LRGs_z06_08 ' + recon + ' recon poly ' + fit][item]
    
    axes[2].scatter(count+delta, x0[beta_code].center-x1[beta_code].center, color=color, alpha=0.5)
    beta_LRG2.append(x0[beta_code].center-x1[beta_code].center)

    # -----------------------------

    if dat == '_pk':
        
        x0 = fits_summaries['LRGs_z08_11 ' + recon + ' recon spline ' + fit][item]
        x1 = fits_summaries['LRGs_z08_11 ' + recon + ' recon poly ' + fit][item]

        axes[3].scatter(count+delta, x0[beta_code].center-x1[beta_code].center, color=color, alpha=0.5)
        beta_LRG3.append(x0[beta_code].center-x1[beta_code].center)

    if dat == '_xi': 
        
        x0 = fits_summaries['ELGsLRGscombined_z08_11 ' + recon + ' recon spline ' + fit][item]
        x1 = fits_summaries['ELGsLRGscombined_z08_11 ' + recon + ' recon poly ' + fit][item]

        axes[3].scatter(count+delta, x0[beta_code].center-x1[beta_code].center, color=color, alpha=0.5)
        beta_LRG3ELG1.append(x0[beta_code].center-x1[beta_code].center)

    # -----------------------------
    
    x0 = fits_summaries['ELGs_z11_16 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['ELGs_z11_16 ' + recon + ' recon poly ' + fit][item]
    
    axes[4].scatter(count+delta, x0[beta_code].center-x1[beta_code].center, color=color, alpha=0.5)
    beta_ELG2.append(x0[beta_code].center-x1[beta_code].center)

    # -----------------------------

    x0 = fits_summaries['QSOs_z08_21 ' + recon + ' recon spline ' + fit][item]
    x1 = fits_summaries['QSOs_z08_21 ' + recon + ' recon poly ' + fit][item]
    
    axes[5].scatter(count+delta, x0[beta_code].center-x1[beta_code].center, color=color, alpha=0.5)
    beta_QSO.append(x0[beta_code].center-x1[beta_code].center)
    
    # -----------------------------

    x0 = fits_summaries['combined fits ' + recon + ' recon spline ' + fit][item if item == ' mean' else str(int(item)+1)]
    x1 = fits_summaries['combined fits ' + recon + ' recon poly ' + fit][item if item == ' mean' else str(int(item)+1)]
    
    axes[6].scatter(count+delta, x0[beta_code_comb].center-x1[beta_code_comb].center, color=color, alpha=0.5)
    beta_combined.append(x0[beta_code_comb].center-x1[beta_code_comb].center)
    

    count += 1 
    
listdat = [beta_BGS, beta_LRG1, beta_LRG2, beta_LRG3, beta_ELG2, beta_QSO, beta_combined]
if dat == '_xi': 
    listdat = [beta_BGS, beta_LRG1, beta_LRG2, beta_LRG3ELG1, beta_ELG2, beta_QSO, beta_combined]

for i, dictt in enumerate(listdat): 
    
    axes[i].fill_between([-1, 26], np.mean(dictt[1:])-np.std(dictt[1:])/5.0, 
            np.mean(dictt[1:])+np.std(dictt[1:])/5.0, alpha=0.1, color='blue') 